In [1]:
import pandas as pd

In [2]:
df_course = pd.read_csv('../../employeeEngagement/prep/courses.csv')

df_chapter = pd.read_csv('../../employeeEngagement/prep/chapters.csv')

df_quiz = pd.read_csv('../../employeeEngagement/prep/quiz.csv')

df_employee = pd.read_csv('../../employeeEngagement/prep/employees.csv')

In [3]:
merged_df_chapter_quiz = pd.merge(
    df_chapter, df_quiz,
    left_on=['id'],
    right_on=['chapterId'],
    how='inner'
)

merged_df_chapter_quiz = merged_df_chapter_quiz.drop(columns=['title_x','title_y','id','id_y'])

def change_score(row):
    if(row['score']>row['totalMarks']):
        return row['totalMarks']
    return row['score']

merged_df_chapter_quiz['score'] = merged_df_chapter_quiz.apply(change_score,axis=1)

display(merged_df_chapter_quiz)

,Unnamed: 0_x,courseId,Unnamed: 0_y,chapterId,totalMarks,totalQuestion,employeeId,quizId,score,completedAt
0,0,1,226,1,17,6,20,1,17,2024-10-07
1,0,1,227,1,11,5,20,2,11,2024-10-07
2,0,1,657,1,17,6,54,1,17,2024-10-07
3,0,1,658,1,11,5,54,2,11,2024-10-07
4,0,1,1191,1,17,6,98,1,4,2024-10-07
...,...,...,...,...,...,...,...,...,...,...
1231,244,50,782,245,23,8,63,367,19,2024-10-07
1232,244,50,1057,245,12,4,87,366,0,2024-10-07
1233,244,50,1058,245,23,8,87,367,23,2024-10-07
1234,244,50,1223,245,12,4,99,366,1,2024-10-07


In [4]:
unique_marks_df = df_quiz.drop_duplicates(subset=['chapterId', 'totalMarks'])

merged_df_course_quiz = pd.merge(
    df_chapter, unique_marks_df,
    left_on=['id'],
    right_on=['chapterId'],
    how='inner'
)

merged_df_course_quiz = merged_df_course_quiz.groupby(['courseId'])['totalMarks'].sum().reset_index(name='total_unique_marks')

display(merged_df_course_quiz.sort_values(by='courseId'))

,courseId,total_unique_marks
0,1,161
1,2,122
2,3,195
3,4,141
4,5,98
5,6,168
6,7,180
7,8,238
8,9,199
9,10,90


In [5]:

merged_df_chapter_quiz = merged_df_chapter_quiz.groupby(['courseId', 'employeeId'])['score'].sum().reset_index(name='total_score')
display(merged_df_chapter_quiz)


,courseId,employeeId,total_score
0,1,14,60
1,1,20,105
2,1,32,36
3,1,54,49
4,1,81,80
...,...,...,...
326,50,60,26
327,50,63,122
328,50,83,35
329,50,87,49


In [6]:
merged_df = pd.merge(
    merged_df_course_quiz, merged_df_chapter_quiz,
    left_on=['courseId'],
    right_on=['courseId'],
    how='inner'
)

merged_df['scorePercentage'] =  (merged_df['total_score'] / merged_df['total_unique_marks'] * 100).round(2)

display(merged_df)


,courseId,total_unique_marks,employeeId,total_score,scorePercentage
0,1,161,14,60,37.27
1,1,161,20,105,65.22
2,1,161,32,36,22.36
3,1,161,54,49,30.43
4,1,161,81,80,49.69
...,...,...,...,...,...
326,50,161,60,26,16.15
327,50,161,63,122,75.78
328,50,161,83,35,21.74
329,50,161,87,49,30.43


In [7]:
final_merge = pd.merge(
    df_employee, merged_df,
    left_on=['id'],
    right_on=['employeeId'],
    how='inner'
)

final_merge = final_merge.drop(columns=['department.1','email','id','Unnamed: 1'])

display(final_merge)

,department,name,team,courseId,total_unique_marks,employeeId,total_score,scorePercentage
0,Business Analysis,Laura Archer,hotel,3,195,23,70,35.90
1,Business Analysis,Laura Archer,hotel,40,134,23,85,63.43
2,Business Analysis,Laura Archer,hotel,45,169,23,17,10.06
3,Business Analysis,Laura Archer,hotel,49,171,23,60,35.09
4,Business Analysis,Amy Powers,bed,4,141,24,1,0.71
...,...,...,...,...,...,...,...,...
326,User Interface (UI) Design,Katie Warren,meet,1,161,98,24,14.91
327,User Interface (UI) Design,Katie Warren,meet,13,137,98,12,8.76
328,User Interface (UI) Design,Katie Warren,meet,19,99,98,22,22.22
329,User Interface (UI) Design,Katie Warren,meet,43,130,98,39,30.00
